In [ ]:
from fastai.vision.all import *
from fastai.data.all import *

In [ ]:
DATASET_NAME = 'MAGNETIC_TILE_SURFACE_DEFECTS'
CODE_NAMES = [0, 1]

PROJ_PATH = Path.cwd().parent
DATASET_PATH = PROJ_PATH/'data'/DATASET_NAME
print(PROJ_PATH)
print(DATASET_PATH)


In [ ]:
train_fpaths = get_image_files(DATASET_PATH/'train_images')
test_fpaths = get_image_files(DATASET_PATH/'test_images')
len(train_fpaths), len(test_fpaths)

In [ ]:
def get_mask_path(img_path): 
    msk_path = Path(str(img_path.parent).replace('_images', '_masks'))/f'{img_path.stem}.png'
    return msk_path

get_mask_path(test_fpaths[0])

In [ ]:
sz = 224
item_tfms_sz = sz
batch_tfms_sz = sz
item_tfms_sz, batch_tfms_sz

In [ ]:
str(Path(train_fpaths[0]).parent.name).startswith('train')

In [ ]:
dls = SegmentationDataLoaders.from_label_func(
    path=DATASET_PATH, 
    fnames=train_fpaths + test_fpaths,
    label_func=get_mask_path,
    codes=CODE_NAMES,
    bs=16, 
    splitter=FuncSplitter(lambda o: str(Path(o).parent.name).startswith('test')),
    item_tfms = Resize(item_tfms_sz),
    batch_tfms =[*aug_transforms(size=batch_tfms_sz), 
                 Normalize.from_stats(*imagenet_stats)]
)

#dls.device = 'cpu' # uncomment to use CPU, might take longer to train

dls.show_batch()
batch = dls.one_batch()
print(batch[0].min(), batch[0].max())
print(batch[1].min(), batch[1].max())

In [ ]:
len(dls.train_ds), len(dls.valid_ds), len(dls.train_ds) + len(dls.valid_ds)

In [ ]:
metrics = [Dice, JaccardCoeff]  

In [ ]:
model_dir = PROJ_PATH/'models'
model_dir.mkdir(exist_ok=True)
model_pickle_fpath = model_dir/'model_pickle_fastai.pkl'
model_pickle_fpath

In [ ]:
learn = unet_learner(dls, resnet34, metrics=metrics)
learn.fine_tune(15)

In [ ]:
learn.show_results()

In [ ]:
loss_val, dice_val, jacc_val = learn.validate()
loss_val, dice_val, jacc_val

In [ ]:
interp = SegmentationInterpretation.from_learner(learn)
interp.plot_top_losses(10,figsize=(10,40))